# Efficient Yelp API Calls
Cameron Peace

### Task

You will use the Yelp API to search your favorite city for a cuisine type of your choice.

Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

Save your notebook, commit the change to your repository and submit the repository URL for this assignment.

### Imports

In [3]:
import pandas as pd
import numpy as np
import json
from yelpapi import YelpAPI
import os
import math
import time
from tqdm.notebook import tqdm_notebook

## Setting Up the API

In [4]:
# loading the api info into a variable
with open('/Users/cameron/.secret/yelp_api.json') as f:
    api_dict = json.load(f)

# confirming
api_dict.keys()

dict_keys(['client-id', 'api-key'])

In [5]:
# Instantiating an api object
yelp_api = YelpAPI(api_dict['api-key'], timeout_s=5)

# confirming
yelp_api

#### API Call Parameters

In [6]:
# terms to use for the api
api_location = 'Seattle, WA'
api_term = 'thai food'

#### Creating a File to Store Results

In [7]:
json_file = '/Users/cameron/Documents/GitHub/CD_Efficient_Yelp_API_Calls/Data/seattle_thai_results'
json_file

'/Users/cameron/Documents/GitHub/CD_Efficient_Yelp_API_Calls/Data/seattle_thai_results'

### Performing an Initial Query

In [10]:
# first query
initial_results = yelp_api.search_query(location=api_location, term=api_term)
# displaying names of keys
initial_results.keys()

dict_keys(['businesses', 'total', 'region'])

In [12]:
# checking total results and current number of results obtained
total_initial_results = initial_results['total']
initial_results_per_page = len(initial_results['businesses'])

print(f'Total Results for this search query: {total_initial_results}')
print(f'Number of Results per Page: {initial_results_per_page}')

Total Results for this search query: 933
Number of Results per Page: 20


<mark><u>**Comment:**</u>

<font color='dodgerblue' size=4><i>
It appears we won't cross the 1,000 results threshold that Yelp allows for free accounts, however, I will still design the functions and loops for the API call based around LP recommendations.
</i></font>

## Creating API Functions and Loops

<mark><u>**Comment:**</u>

<font color='dodgerblue' size=4><i>
This function checks if a JSON file to capture the results of the API call exists.  If not, it creates both the folder and file, adding an empty list to the JSON file. It also has an option for deleting the existing file if desired.
</i></font>

In [13]:
# function used to create a new json and folder, or confirm existing file
def create_json_file(_file, delete_if_exists=False): 
    if os.path.isfile(_file) == True:
        if delete_if_exists == True:
            print(f"** {_file} already exists. Deleting previous file...")
            os.remove(_file)
            create_json_file(_file, delete_if_exists=False)
        else:
            print(f"** {_file} already exists.")            
    else:
        print(f"** {_file} not found. Saving empty list to new file.")
        folder = os.path.dirname(_file)
        if len(folder) > 0:
            os.makedirs(folder, exist_ok=True)
        with open(_file, 'w') as f:
            json.dump([], f)  

In [14]:
# create a new empty json file (exist the previous if it exists)
create_json_file(json_file, delete_if_exists=True)

# read the file and save the results to a variable
with open(json_file,'r') as f:
    previous_results = json.load(f)

# get the number of the results in the file and display
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

** /Users/cameron/Documents/GitHub/CD_Efficient_Yelp_API_Calls/Data/seattle_thai_results already exists. Deleting previous file...
** /Users/cameron/Documents/GitHub/CD_Efficient_Yelp_API_Calls/Data/seattle_thai_results not found. Saving empty list to new file.
- 0 previous results found.


<mark><u>**Comment:**</u>

<font color='dodgerblue' size=4><i>
We've used our custom function to check for the file, it found the file we had created in our initial check.  It then deleted the existing file and created a new one, checking for the number of results contained within (so it makes sense for the file to have 0 previous results).
</i></font>

### Using the API

In [15]:
# initial api call to get number of results
results = yelp_api.search_query(location=api_location,
                                term=api_term,
                               offset=n_results)

# getting total number of results and number of results per page
total_results = results['total']
results_per_page = len(results['businesses'])

# calculating the number of pages
n_pages = math.ceil((results['total'] - n_results)/ results_per_page)
n_pages

47

<mark><u>**Comment:**</u>

<font color='dodgerblue' size=4><i>
We will need 47 pages to capture all 933 results
</i></font>

In [16]:
# using a loop to fill the JSON file with repeated API calls
# using tqdm in order to display a progress bar

for i in tqdm_notebook(range(1, n_pages+1)):
    with open(json_file, 'r') as f:
        previous_results = json.load(f)
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break

    results = yelp_api.search_query(location=api_location,
                                    term=api_term, 
                                    offset=n_results)
    
    previous_results.extend(results['businesses'])
    
    with open(json_file,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/47 [00:00<?, ?it/s]

<mark><u>**Comment:**</u>

<font color='dodgerblue' size=4><i>
Looks like everything worked!  We made 47 API calls and should have received 933 entries for thai food in Seattle, WA from the Yelp database.
</i></font>

## Viewing Results of API calls

In [18]:
final_df = pd.read_json(json_file)
display(final_df.head(), final_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 933 entries, 0 to 932
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             933 non-null    object 
 1   alias          933 non-null    object 
 2   name           933 non-null    object 
 3   image_url      933 non-null    object 
 4   is_closed      933 non-null    bool   
 5   url            933 non-null    object 
 6   review_count   933 non-null    int64  
 7   categories     933 non-null    object 
 8   rating         933 non-null    float64
 9   coordinates    933 non-null    object 
 10  transactions   933 non-null    object 
 11  price          811 non-null    object 
 12  location       933 non-null    object 
 13  phone          933 non-null    object 
 14  display_phone  933 non-null    object 
 15  distance       933 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 110.4+ KB


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,Aj1oSIHJwbteXYfYquzUOA,bangrak-market-seattle,Bangrak Market,https://s3-media3.fl.yelpcdn.com/bphoto/E3zqsK...,False,https://www.yelp.com/biz/bangrak-market-seattl...,1090,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 47.614263, 'longitude': -122.346387}",[delivery],$$,"{'address1': '2319 2nd Ave', 'address2': None,...",+12067357352,(206) 735-7352,1227.074663
1,zS4wWolhzmKtR5rgnpw_3A,fern-thai-capitol-hill-seattle-3,Fern Thai Capitol Hill,https://s3-media4.fl.yelpcdn.com/bphoto/GTDMvM...,False,https://www.yelp.com/biz/fern-thai-capitol-hil...,90,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.5,"{'latitude': 47.61313011320635, 'longitude': -...",[],$$,"{'address1': '1400 10th Ave', 'address2': '', ...",+12068586655,(206) 858-6655,817.417092
2,bBDeZ3QkljIbPPeQ__Xq-w,noi-thai-cuisine-downtown-seattle-seattle,Noi Thai Cuisine - Downtown Seattle,https://s3-media4.fl.yelpcdn.com/bphoto/b9bGi5...,False,https://www.yelp.com/biz/noi-thai-cuisine-down...,1117,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.0,"{'latitude': 47.606947561992335, 'longitude': ...","[delivery, pickup]",$$,"{'address1': '1303 1st Ave', 'address2': '', '...",+12067878444,(206) 787-8444,1038.614352
3,Ic8vm_azp9sKVBp9BRBufg,sisi-kay-thai-eatery-and-bar-seattle-5,Sisi Kay Thai Eatery & Bar,https://s3-media2.fl.yelpcdn.com/bphoto/sdQO6D...,False,https://www.yelp.com/biz/sisi-kay-thai-eatery-...,378,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.5,"{'latitude': 47.66161, 'longitude': -122.33855}","[pickup, delivery]",$$,"{'address1': '1612 N 45th St', 'address2': Non...",+12066594382,(206) 659-4382,5291.861794
4,h7Dbi62QzPRXtqmlwUFhuA,bai-tong-thai-street-cafe-seattle,Bai Tong Thai Street Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/QuChdb...,False,https://www.yelp.com/biz/bai-tong-thai-street-...,659,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.0,"{'latitude': 47.61393, 'longitude': -122.31719}","[delivery, pickup, restaurant_reservation]",$$,"{'address1': '1121 E Pike St', 'address2': '',...",+12067878448,(206) 787-8448,963.915492


None

<mark><u>**Comment:**</u>

<font color='dodgerblue' size=4><i>
Looks good here, cleaning and analysis is beyond the scope of this notebook so we'll leave it at this.
</i></font>